In [1]:
from utilspatches import *
from upaths import tilenames, names_at_tilling_V1
from upaths import TILES12_DPATH

In [2]:
names_at_patching_V1 = ['edem_demw84','ldtm','pdem','tdem_dem',
                        'egm08x','egm96x','s1x','s2x','tdem_hem','esawc','lwm',
                       'tdem_dem_fw','tdem_dem_mw']

len(names_at_patching_V1)

13

In [11]:
# that code that start running from genvert foward or others foward 

In [3]:
vrtvars = names_at_patching_V1
i_wdir = TILES12_DPATH
ps = 256

In [3]:
from glob import glob

In [19]:
path = "/media/ljp238/12TBWolf/RSPROX/TILES12_patches256/S01W063"

In [ ]:
subdirs = [os.path.join(path,i) for i in os.listdir(path)]
subdirs = [i for i in subdirs if os.path.isdir(i)]
for subdir in subdirs:
    subdirs_tif = glob(os.path.join(subdir, '*.tif'))
    print(f"Number of .tif files in {subdir}: {len(subdirs_tif)}")

# make each subdirect column of df, and all the files as row, return a df

In [23]:
import os
import pandas as pd
from glob import glob
from upaths import tilenames_full # tilenames_train  instead
from upaths import PATCHE256_DPATH,CONFIG_PATCHE256_DPATH,columns_to_check
from concurrent.futures import ProcessPoolExecutor

def create_tif_dataframe(path):
  subdirs = [os.path.join(path, i) for i in os.listdir(path)]
  subdirs = [i for i in subdirs if os.path.isdir(i)]
  data = {}
  for subdir in subdirs:
      subdirs_tif = glob(os.path.join(subdir, '*.tif'))
      subdir_name = os.path.basename(subdir)
      data[subdir_name] = subdirs_tif
  df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))
  return df

In [ ]:
for tilename in tilenames_full:
    directory_path = os.path.join(PATCHE256_DPATH, tilename)
    print(f"Processing directory: {directory_path}")
    #csvpath = os.path.join(CONFIG_PATCHE256_DPATH, f'{tilename}_patches_count_null.csv')
    #df = create_tif_dataframe(directory_path)
    #print(df.shape)

In [ ]:
import os
import pandas as pd
from osgeo import gdal
import rasterio

from upaths import tilenames_full # tilenames_train  instead
from upaths import PATCHE256_DPATH,CONFIG_PATCHE256_DPATH,columns_to_check
from concurrent.futures import ProcessPoolExecutor


import os
import pandas as pd
import rasterio
from concurrent.futures import ProcessPoolExecutor

# Assuming these are defined elsewhere in your code
# from upaths import tilenames_full, PATCHE256_DPATH, CONFIG_PATCHE256_DPATH, columns_to_check

def create_dataframe_from_directory(directory):
  # Dictionary to hold lists of file paths for each subdirectory
  data = {}

  # Walk through the directory
  for root, dirs, files in os.walk(directory):
      # Get the relative path of the current directory
      relative_path = os.path.relpath(root, directory)

      # Skip the root directory itself
      if relative_path == ".":
          continue

      # Initialize a list for the current subdirectory if not already present
      if relative_path not in data:
          data[relative_path] = []

      # Add full paths of .tif files to the list
      for file in files:
          if file.endswith('.tif'):
              full_path = os.path.join(root, file)
              data[relative_path].append(full_path)

  # Create a DataFrame from the dictionary
  if data:
      df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))
  else:
      df = pd.DataFrame()

  return df

def count_nulls_in_rasters(df, columns):
  # Add new columns for null counts
  for column in columns:
      if column in df.columns:
          null_counts = []
          for file_path in df[column].dropna():
              try:
                  with rasterio.open(file_path) as src:
                      # Read the data
                      data = src.read(1)
                      # Count nulls (assuming nulls are represented by NaN or a specific nodata value)
                      nodata_value = src.nodata
                      if nodata_value is not None:
                          null_count = (data == nodata_value).sum()
                      else:
                          null_count = pd.isnull(data).sum()
                      null_counts.append(null_count)
              except Exception as e:
                  print(f"Error processing file {file_path}: {e}")
                  null_counts.append(None)

          # Add the null counts to the DataFrame
          df[f'{column}_null_count'] = pd.Series(null_counts)

  return df

def count_nulls_by_variablelist(directory_path, columns_to_check, csvpath):
  df = create_dataframe_from_directory(directory_path)
  if df.empty:
      print(f"No .tif files found in {directory_path}")
  else:
      print(df.head())
      df_with_null_counts = count_nulls_in_rasters(df, columns_to_check)
      df_with_null_counts.to_csv(csvpath, index=False)
      print(f'Processed {directory_path} and saved to {csvpath}')

if __name__ == '__main__':
  with ProcessPoolExecutor() as PEX:
      for tilename in tilenames_full:
          directory_path = os.path.join(PATCHE256_DPATH, tilename)
          print(f"Processing directory: {directory_path}")
          csvpath = os.path.join(CONFIG_PATCHE256_DPATH, f'{tilename}_patches_count_null.csv')

          PEX.submit(count_nulls_by_variablelist, directory_path, columns_to_check, csvpath)
        

        
       
        
        

In [ ]:

# Count nulls and add new columns






In [ ]:
# in getting DFRA # geotile has a way
# water as 0, all other elevation filled 
# all features filled 
# there's no gaps in the data 

In [ ]:
# get nodata  and print 
# all values in raster without nodata 
# filter any below or equall -99 and set to nodata 
# filter any above or equall to 10000 and set to nodata 
# remove old no value and replace it with a new one -9999. float 
# write the dataset
# get nodata and print it 


In [17]:
import os
import pandas as pd
import rasterio
from concurrent.futures import ThreadPoolExecutor #,ProcessPoolExecutor

def create_dataframe_from_directory(directory):
  # Dictionary to hold lists of file paths for each subdirectory
  data = {}

  # Walk through the directory
  for root, dirs, files in os.walk(directory):
      # Get the relative path of the current directory
      relative_path = os.path.relpath(root, directory)

      # Skip the root directory itself
      if relative_path == ".":
          continue

      # Initialize a list for the current subdirectory if not already present
      if relative_path not in data:
          data[relative_path] = []

      # Add full paths of .tif files to the list
      for file in files:
          if file.endswith('.tif'):
              full_path = os.path.join(root, file)
              data[relative_path].append(full_path)

  # Create a DataFrame from the dictionary
  df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))

  return df

def count_nulls_in_file(file_path):
  with rasterio.open(file_path) as src:
      # Read the data
      data = src.read(1)
      # Count nulls (assuming nulls are represented by NaN or a specific nodata value)
      nodata_value = src.nodata
      if nodata_value is not None:
          null_count = (data == nodata_value).sum()
      else:
          null_count = pd.isnull(data).sum()
  return null_count

def count_nulls_in_rasters(df, columns):
  # Add new columns for null counts
  for column in columns:
      null_counts = []
      file_paths = df[column].dropna().tolist()

      # Use ThreadPoolExecutor for parallel processing
      with ThreadPoolExecutor() as executor:
          null_counts = list(executor.map(count_nulls_in_file, file_paths))

      # Add the null counts to the DataFrame
      df[f'{column}_null_count'] = pd.Series(null_counts)

  return df

# Example usage
df = create_dataframe_from_directory(directory_path)

# Specify the columns to check for nulls
columns_to_check = ['ldtm', 'pdem', 'tdem_dem', 'tdem_dem_fw']
columns_to_check_x =  [f'{col}_null_count' for col in columns_to_check]
# Count nulls and add new columns
df_with_null_counts = count_nulls_in_rasters(df, columns_to_check)

In [20]:
df_with_null_counts[columns_to_check_x[1]].value_counts()

pdem_null_count
0        1088
65536     108
22272      34
8960       32
55040      32
28187       1
58607       1
Name: count, dtype: int64